<a href="https://colab.research.google.com/github/lejejefr/ECM_2526_FinalProject/blob/main/Sripts/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ce script sert à entrainer notre modèle et à l'utiliser s'il est déjà entrainé
import pandas as pd
import numpy as np

In [14]:
def area(lat1, lat2, lon1, lon2, R = 6378):
    lat1 = np.pi * lat1 / 180.0
    lat2 = np.pi * lat2 / 180.0
    lon1 = np.pi * lon1 / 180.0
    lon2 = np.pi * lon2 / 180.0

    h = max(np.sin(lat1), np.sin(lat2)) - min(np.sin(lat1), np.sin(lat2))
    r = (max(lon1, lon2) - min(lon1,lon2)) / (2 * np.pi)

    return 2 * np.pi * (R ** 2) * h * r

print(area(36.619446,42.022548,-91.686566,-87.525759))

215429.1889977955
